---
# Imports

In [1]:
# imports.py
from imports import *
from sklearn.datasets import make_blobs

# plotting magic
%matplotlib inline

# custom mods
import tidy
import model_h
import wrangle

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


---
# Wrangle

In [2]:
df=wrangle.wrangle_df()
# df.info()
# # model_h.print_btcusd(df)
# df.head()

cached csv


In [3]:
df['atr'] = talib.ATR(df.high, df.low, df.close, 14)

In [4]:
df=df.loc['2021-2':]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 478 entries, 2021-02-01 to 2022-05-24
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   open                          478 non-null    float64
 1   high                          478 non-null    float64
 2   low                           478 non-null    float64
 3   close                         478 non-null    float64
 4   volume                        478 non-null    float64
 5   fwd_log_ret                   478 non-null    float64
 6   fwd_ret                       478 non-null    float64
 7   fwd_pct_chg                   478 non-null    float64
 8   fwd_close_positive            478 non-null    float64
 9   macd                          478 non-null    float64
 10  signal                        478 non-null    float64
 11  histo                         478 non-null    float64
 12  cross                         478 non-null   

---
## Train / Test Split

In [6]:
train_unscaled, val_unscaled=tidy.split_ii(df)


# # train_ii, test=tidy.split_ii(df)
# # train_ii.info(),test.info()


---
## Scaler

In [7]:
transformer = RobustScaler().fit(train_unscaled)

In [8]:
train=transformer.transform(train_unscaled)
val=transformer.transform(val_unscaled)

In [9]:
train=pd.DataFrame(train,columns=train_unscaled.columns)
val=pd.DataFrame(val,columns=val_unscaled.columns)

In [10]:
# def scaler_fit(X_train):
# 	for col in X_train.columns:
# 		if X_train[col].dtype == 'float64':
# 			xs=np.array(X_train[[col]]).reshape(-1, 1)
# 			X_train.loc[:,col]=RobustScaler().fit_transform(xs)
# 			print(col)
# 	return X_train


In [11]:
# def scaler_tran(df):
# 	for col in df.columns:
# 		if df[col].dtype == 'float64':
# 			xs=np.array(df[[col]]).reshape(-1, 1)
# 			X_train.loc[:,col]=RobustScaler().transform(xs)
# 			print(col)
# 	return X_train


In [12]:
# X_train=scaler_fit(X_train)


---
## Xy Split

In [13]:
X_train=train[['open', 'high', 'low', 'close', 'volume', 'obv_close_product','atr', 'macd', 'signal', 'histo', 'sentiment']]
X_val=val[['open', 'high', 'low', 'close', 'volume', 'obv_close_product','atr', 'macd', 'signal', 'histo', 'sentiment']]

In [14]:
y_train=train[[ 'fwd_ret']]
y_val=val[[ 'fwd_ret']]

---
# RFE Regression

In [15]:
lm = LinearRegression()

In [16]:
rfe = RFE(lm, n_features_to_select=8)

In [17]:
# Transforming data using RFE
X_rfe = rfe.fit_transform(X_train,y_train.fwd_ret)  
mask=rfe.support_
rfe_features = X_train.loc[:,mask].columns.tolist()

X_train=pd.DataFrame(X_rfe,columns=rfe_features)
X_train.head()


,open,high,low,close,obv_close_product,signal,histo,sentiment
0,-0.904588,-0.832396,-0.874849,-0.876747,-0.010531,0.237558,-0.402388,0.034693
1,-0.875865,-0.749787,-0.796808,-0.741448,0.261047,0.224587,-0.171452,0.435904
2,-0.740176,-0.645872,-0.658752,-0.606925,0.605632,0.231445,0.138414,-0.140106
3,-0.605402,-0.572890,-0.597060,-0.644367,0.395730,0.247012,0.274486,0.325468
4,-0.642697,-0.596929,-0.572958,-0.560830,0.665091,0.273133,0.439408,-1.314773


In [18]:
# var_ranks = rfe.ranking_
# var_names = X_rfe.columns.tolist()

# pd.DataFrame({'Var': var_names, 'Rank': var_ranks})


In [19]:
X_val=X_val[rfe_features]

---
# Regression Model

In [20]:

line=lm.fit(X_train,y_train)


In [21]:

y_train['reg'] = lm.predict(X_train)

y_train.to_csv('reg_train_ii.csv')


/var/folders/mk/w2hz4wxx7xn5hr_56wz536gh0000gn/T/ipykernel_57034/2063499806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['reg'] = lm.predict(X_train)


In [22]:

rmse_train = metrics.mean_squared_error(y_train.fwd_ret, y_train.reg)**(1/2)


In [23]:

y_val['reg'] = lm.predict(X_val)

y_val.to_csv('reg_val_ii.csv')


/var/folders/mk/w2hz4wxx7xn5hr_56wz536gh0000gn/T/ipykernel_57034/1548275373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_val['reg'] = lm.predict(X_val)


In [24]:

rmse_validate = metrics.mean_squared_error(y_val.fwd_ret, y_val.reg)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for OLS using LinearRegression
Training/In-Sample:  0.8935154573257269 
Validation/Out-of-Sample:  0.7612564719979116
